In [1]:
from tqdm import tqdm
import os
os.environ['TF_ENABLE_CONTROL_FLOW_V2'] = '1'

import tensorflow as tf
#import tensorflow.nn.rnn_cell as rnn
import numpy as np
#import tensorflow.keras as keras
#from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend as K

In [2]:
def buildLstmLayer(x):
    """Build the lstm layer.

    Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
    """
    num_units=64
  
    inputs = x[0]
    initial_state = x[1]
    lstm_layers = tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn1',state_is_tuple=False)
  # Assume the input is sized as [batch, time, input_size], then we're going
  # to transpose to be time-majored.
    transposed_inputs = tf.transpose(
      inputs, perm=[1, 0, 2])
    outputs, state = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      transposed_inputs,
      initial_state=initial_state,
      dtype='float32',
      time_major=True)
    #unstacked_outputs = tf.unstack(outputs, axis=0)
    return [outputs,state]

In [3]:
num_units=64

inputs = tf.keras.layers.Input(shape=(1,3),name='input')
inputs_state = tf.keras.layers.Input(shape=(num_units*2),name='input_state')
'''lstm_layers = tf.lite.experimental.nn.TFLiteLSTMCell(
            64, forget_bias=0, name='rnn1')#,state_is_tuple=False)
transposed_inputs = tf.transpose(
      inputs, perm=[1, 0, 2])
outputs, new_state = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      inputs=transposed_inputs,
      #initial_state = inputs_state,
      dtype='float32',
      time_major=True)
unstacked_outputs = tf.unstack(outputs, axis=0)'''

output,new_state = tf.keras.layers.Lambda(buildLstmLayer)([inputs,inputs_state])
new_state = tf.keras.layers.Flatten(name='state_out')(new_state)
output =tf.keras.layers.Dense(1,activation=tf.nn.relu, name='output')(output)

model = tf.keras.models.Model(inputs=[inputs,inputs_state],
                                      outputs=[output,new_state])
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.summary()
'''
def buildLstmLayer(inputs, num_layers, num_units):
  """Build the lstm layer.

  Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
  """
  lstm_cells = []
  for i in range(num_layers):
    lstm_cells.append(
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i)))
  #lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
  lstm_layers = tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i))
  # Assume the input is sized as [batch, time, input_size], then we're going
  # to transpose to be time-majored.
  transposed_inputs = tf.transpose(
      inputs, perm=[1, 0, 2])
  outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      transposed_inputs,
      dtype='float32',
      time_major=True)
  unstacked_outputs = tf.unstack(outputs, axis=0)
  return unstacked_outputs[-1]

tf.reset_default_graph()
model = tf.keras.models.Sequential([
  #tf.keras.layers.Input(shape=(3, 3), name='input'),
  tf.keras.layers.Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 64},input_shape=(1,3)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation=tf.nn.relu, name='output')
])
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.summary()'''

W0731 08:24:03.270919 13132 rnn_cell.py:246] <tensorflow.lite.experimental.examples.lstm.rnn_cell.TFLiteLSTMCell object at 0x000002BDC35F13C8>: Using a concatenated state is slower and will soon be deprecated.  Use state_is_tuple=True.
W0731 08:24:03.356688 13132 deprecation.py:506] From C:\Users\Guo\Anaconda3\envs\tf_1_14\lib\site-packages\tensorflow\lite\experimental\examples\lstm\rnn_cell.py:320: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 08:24:03.358683 13132 deprecation.py:506] From C:\Users\Guo\Anaconda3\envs\tf_1_14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 1, 3)]       0                                            
__________________________________________________________________________________________________
input_state (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 [(1, None, 64), (Non 17408       input[0][0]                      
                                                                 input_state[0][0]                
__________________________________________________________________________________________________
output (Dense)                  (1, None, 1)         65          lambda[0][0]                 

'\ndef buildLstmLayer(inputs, num_layers, num_units):\n  """Build the lstm layer.\n\n  Args:\n    inputs: The input data.\n    num_layers: How many LSTM layers do we want.\n    num_units: The unmber of hidden units in the LSTM cell.\n  """\n  lstm_cells = []\n  for i in range(num_layers):\n    lstm_cells.append(\n        tf.lite.experimental.nn.TFLiteLSTMCell(\n            num_units, forget_bias=0, name=\'rnn{}\'.format(i)))\n  #lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)\n  lstm_layers = tf.lite.experimental.nn.TFLiteLSTMCell(\n            num_units, forget_bias=0, name=\'rnn{}\'.format(i))\n  # Assume the input is sized as [batch, time, input_size], then we\'re going\n  # to transpose to be time-majored.\n  transposed_inputs = tf.transpose(\n      inputs, perm=[1, 0, 2])\n  outputs, _ = tf.lite.experimental.nn.dynamic_rnn(\n      lstm_layers,\n      transposed_inputs,\n      dtype=\'float32\',\n      time_major=True)\n  unstacked_outputs = tf.unstack(outputs, axis=0)\n 

In [4]:
def buildLstmLayer(inputs, num_layers, num_units):
  """Build the lstm layer.

  Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
  """
  lstm_cells = []
  for i in range(num_layers):
    lstm_cells.append(
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i)))
  #lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
  lstm_layers = tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i))
  # Assume the input is sized as [batch, time, input_size], then we're going
  # to transpose to be time-majored.
  transposed_inputs = tf.transpose(
      inputs, perm=[1, 0, 2])
  outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      transposed_inputs,
      dtype='float32',
      time_major=True)
  unstacked_outputs = tf.unstack(outputs, axis=0)
  return unstacked_outputs[-1]

tf.reset_default_graph()
model = tf.keras.models.Sequential([
  #tf.keras.layers.Input(shape=(3, 3), name='input'),
  tf.keras.layers.Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 64},input_shape=(1,3)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation=tf.nn.relu, name='output')
])
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
model.summary()

W0730 09:04:15.195551 20020 deprecation.py:506] From C:\Users\Guo\Anaconda3\envs\tf_1_14\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0730 09:04:15.268379 20020 deprecation.py:506] From C:\Users\Guo\Anaconda3\envs\tf_1_14\lib\site-packages\tensorflow\lite\experimental\examples\lstm\rnn_cell.py:320: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 64)                17408     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total params: 17,473
Trainable params: 17,473
Non-trainable params: 0
_________________________________________________________________


In [6]:
re_train=True
time_steps=1
n_input =3
num_units=64
n_classes=1
lr =0.00001
logFile = './traing.txt'

In [7]:



data_x=np.loadtxt('./TapData/data_raw_100Hz.csv',delimiter=",")
data_y=np.loadtxt('./TapData/data_100Hz_y.csv',delimiter=",")
preY = np.loadtxt('./TapData/data_100Hz_y.csv',delimiter=",")

datay=data_y
data_y=data_y.reshape(len(data_y),1)
data_x=data_x[:,(6-n_input):6].reshape((len(data_x),3))
data_x=data_x.reshape([len(data_x),1,n_input])
data_state = np.zeros([1,num_units*2],dtype='float')
#taps_label = detect(data_y,raw=data_x)

loss_weight = preY*len(preY)/preY.sum()+datay*len(datay)/datay.sum()+1


In [ ]:
for i in range (len(data_x)):
    

In [10]:
model.fit(data_x, data_y, epochs=5,batch_size=1,sample_weight=loss_weight)

Epoch 1/5


  8026/512141 [..............................] - ETA: 22:55:40 - loss: 0.0000e+00 - acc: 1.000 - ETA: 50:16 - loss: 3.1428 - acc: 0.9444      - ETA: 30:19 - loss: 2.2934 - acc: 0.95 - ETA: 23:29 - loss: 1.9849 - acc: 0.96 - ETA: 20:12 - loss: 1.4694 - acc: 0.97 - ETA: 18:19 - loss: 2.0412 - acc: 0.96 - ETA: 17:02 - loss: 1.6923 - acc: 0.97 - ETA: 16:08 - loss: 1.8582 - acc: 0.96 - ETA: 15:27 - loss: 2.1619 - acc: 0.96 - ETA: 14:56 - loss: 2.0775 - acc: 0.96 - ETA: 14:30 - loss: 1.8618 - acc: 0.96 - ETA: 14:07 - loss: 1.8123 - acc: 0.96 - ETA: 13:51 - loss: 1.8976 - acc: 0.96 - ETA: 13:37 - loss: 1.8566 - acc: 0.96 - ETA: 13:25 - loss: 1.8249 - acc: 0.96 - ETA: 13:14 - loss: 1.7974 - acc: 0.96 - ETA: 13:05 - loss: 1.8621 - acc: 0.96 - ETA: 13:01 - loss: 1.8438 - acc: 0.96 - ETA: 12:54 - loss: 1.9015 - acc: 0.96 - ETA: 12:48 - loss: 1.8757 - acc: 0.96 - ETA: 12:42 - loss: 1.7812 - acc: 0.96 - ETA: 12:37 - loss: 2.0374 - acc: 0.96 - ETA: 12:32 - loss: 1.9440 - acc: 0.96 - ETA: 12:28 - los

 32372/512141 [>.............................] - ETA: 10:29 - loss: 1.9825 - acc: 0.96 - ETA: 10:29 - loss: 1.9793 - acc: 0.96 - ETA: 10:29 - loss: 1.9761 - acc: 0.96 - ETA: 10:29 - loss: 1.9775 - acc: 0.96 - ETA: 10:29 - loss: 1.9789 - acc: 0.96 - ETA: 10:29 - loss: 1.9781 - acc: 0.96 - ETA: 10:29 - loss: 1.9773 - acc: 0.96 - ETA: 10:29 - loss: 1.9765 - acc: 0.96 - ETA: 10:29 - loss: 1.9780 - acc: 0.96 - ETA: 10:29 - loss: 1.9794 - acc: 0.96 - ETA: 10:29 - loss: 1.9785 - acc: 0.96 - ETA: 10:29 - loss: 1.9776 - acc: 0.96 - ETA: 10:29 - loss: 1.9789 - acc: 0.96 - ETA: 10:29 - loss: 1.9805 - acc: 0.96 - ETA: 10:29 - loss: 1.9818 - acc: 0.96 - ETA: 10:28 - loss: 1.9832 - acc: 0.96 - ETA: 10:28 - loss: 1.9870 - acc: 0.96 - ETA: 10:28 - loss: 1.9838 - acc: 0.96 - ETA: 10:28 - loss: 1.9829 - acc: 0.96 - ETA: 10:28 - loss: 1.9841 - acc: 0.96 - ETA: 10:28 - loss: 1.9810 - acc: 0.96 - ETA: 10:28 - loss: 1.9825 - acc: 0.96 - ETA: 10:28 - loss: 1.9840 - acc: 0.96 - ETA: 10:28 - loss: 1.9831 - acc

 40497/512141 [=>............................] - ETA: 10:18 - loss: 2.0003 - acc: 0.96 - ETA: 10:18 - loss: 2.0048 - acc: 0.96 - ETA: 10:18 - loss: 2.0058 - acc: 0.96 - ETA: 10:17 - loss: 2.0103 - acc: 0.96 - ETA: 10:17 - loss: 2.0082 - acc: 0.96 - ETA: 10:18 - loss: 2.0113 - acc: 0.96 - ETA: 10:18 - loss: 2.0107 - acc: 0.96 - ETA: 10:18 - loss: 2.0120 - acc: 0.96 - ETA: 10:18 - loss: 2.0133 - acc: 0.96 - ETA: 10:18 - loss: 2.0161 - acc: 0.96 - ETA: 10:17 - loss: 2.0153 - acc: 0.96 - ETA: 10:17 - loss: 2.0129 - acc: 0.96 - ETA: 10:17 - loss: 2.0122 - acc: 0.96 - ETA: 10:17 - loss: 2.0148 - acc: 0.96 - ETA: 10:17 - loss: 2.0157 - acc: 0.96 - ETA: 10:17 - loss: 2.0132 - acc: 0.96 - ETA: 10:17 - loss: 2.0123 - acc: 0.96 - ETA: 10:17 - loss: 2.0132 - acc: 0.96 - ETA: 10:17 - loss: 2.0107 - acc: 0.96 - ETA: 10:17 - loss: 2.0116 - acc: 0.96 - ETA: 10:17 - loss: 2.0092 - acc: 0.96 - ETA: 10:17 - loss: 2.0085 - acc: 0.96 - ETA: 10:17 - loss: 2.0061 - acc: 0.96 - ETA: 10:17 - loss: 2.0037 - acc

 48680/512141 [=>............................] - ETA: 10:06 - loss: 2.0249 - acc: 0.96 - ETA: 10:06 - loss: 2.0242 - acc: 0.96 - ETA: 10:06 - loss: 2.0264 - acc: 0.96 - ETA: 10:06 - loss: 2.0244 - acc: 0.96 - ETA: 10:06 - loss: 2.0223 - acc: 0.96 - ETA: 10:06 - loss: 2.0217 - acc: 0.96 - ETA: 10:06 - loss: 2.0196 - acc: 0.96 - ETA: 10:06 - loss: 2.0204 - acc: 0.96 - ETA: 10:06 - loss: 2.0198 - acc: 0.96 - ETA: 10:06 - loss: 2.0233 - acc: 0.96 - ETA: 10:06 - loss: 2.0212 - acc: 0.96 - ETA: 10:06 - loss: 2.0205 - acc: 0.96 - ETA: 10:05 - loss: 2.0226 - acc: 0.96 - ETA: 10:05 - loss: 2.0219 - acc: 0.96 - ETA: 10:05 - loss: 2.0198 - acc: 0.96 - ETA: 10:05 - loss: 2.0192 - acc: 0.96 - ETA: 10:05 - loss: 2.0172 - acc: 0.96 - ETA: 10:05 - loss: 2.0155 - acc: 0.96 - ETA: 10:05 - loss: 2.0190 - acc: 0.96 - ETA: 10:05 - loss: 2.0198 - acc: 0.96 - ETA: 10:05 - loss: 2.0191 - acc: 0.96 - ETA: 10:05 - loss: 2.0254 - acc: 0.96 - ETA: 10:05 - loss: 2.0260 - acc: 0.96 - ETA: 10:05 - loss: 2.0241 - acc

...

 28783/512141 [>.............................] - ETA: 11:47 - loss: 2.0943 - acc: 0.96 - ETA: 11:47 - loss: 2.0910 - acc: 0.96 - ETA: 11:47 - loss: 2.0928 - acc: 0.96 - ETA: 11:47 - loss: 2.0923 - acc: 0.96 - ETA: 11:47 - loss: 2.0890 - acc: 0.96 - ETA: 11:47 - loss: 2.0908 - acc: 0.96 - ETA: 11:47 - loss: 2.0927 - acc: 0.96 - ETA: 11:47 - loss: 2.0969 - acc: 0.96 - ETA: 11:47 - loss: 2.0936 - acc: 0.96 - ETA: 11:47 - loss: 2.0928 - acc: 0.96 - ETA: 11:47 - loss: 2.0921 - acc: 0.96 - ETA: 11:47 - loss: 2.0913 - acc: 0.96 - ETA: 11:47 - loss: 2.0882 - acc: 0.96 - ETA: 11:47 - loss: 2.0876 - acc: 0.96 - ETA: 11:47 - loss: 2.0844 - acc: 0.96 - ETA: 11:47 - loss: 2.0813 - acc: 0.96 - ETA: 11:47 - loss: 2.0805 - acc: 0.96 - ETA: 11:47 - loss: 2.0772 - acc: 0.96 - ETA: 11:47 - loss: 2.0791 - acc: 0.96 - ETA: 11:47 - loss: 2.0782 - acc: 0.96 - ETA: 11:47 - loss: 2.0799 - acc: 0.96 - ETA: 11:47 - loss: 2.0791 - acc: 0.96 - ETA: 11:47 - loss: 2.0783 - acc: 0.96 - ETA: 11:46 - loss: 2.0825 - acc

 35839/512141 [=>............................] - ETA: 11:42 - loss: 2.0888 - acc: 0.96 - ETA: 11:42 - loss: 2.0883 - acc: 0.96 - ETA: 11:42 - loss: 2.0880 - acc: 0.96 - ETA: 11:42 - loss: 2.0895 - acc: 0.96 - ETA: 11:42 - loss: 2.0890 - acc: 0.96 - ETA: 11:42 - loss: 2.0905 - acc: 0.96 - ETA: 11:42 - loss: 2.0881 - acc: 0.96 - ETA: 11:42 - loss: 2.0895 - acc: 0.96 - ETA: 11:42 - loss: 2.0869 - acc: 0.96 - ETA: 11:42 - loss: 2.0883 - acc: 0.96 - ETA: 11:42 - loss: 2.0858 - acc: 0.96 - ETA: 11:42 - loss: 2.0834 - acc: 0.96 - ETA: 11:42 - loss: 2.0829 - acc: 0.96 - ETA: 11:42 - loss: 2.0823 - acc: 0.96 - ETA: 11:41 - loss: 2.0817 - acc: 0.96 - ETA: 11:41 - loss: 2.0793 - acc: 0.96 - ETA: 11:41 - loss: 2.0807 - acc: 0.96 - ETA: 11:41 - loss: 2.0800 - acc: 0.96 - ETA: 11:41 - loss: 2.0776 - acc: 0.96 - ETA: 11:41 - loss: 2.0789 - acc: 0.96 - ETA: 11:41 - loss: 2.0841 - acc: 0.96 - ETA: 11:41 - loss: 2.0854 - acc: 0.96 - ETA: 11:41 - loss: 2.0829 - acc: 0.96 - ETA: 11:41 - loss: 2.0842 - acc

 42899/512141 [=>............................] - ETA: 11:34 - loss: 2.1115 - acc: 0.96 - ETA: 11:33 - loss: 2.1093 - acc: 0.96 - ETA: 11:33 - loss: 2.1088 - acc: 0.96 - ETA: 11:33 - loss: 2.1131 - acc: 0.96 - ETA: 11:33 - loss: 2.1127 - acc: 0.96 - ETA: 11:33 - loss: 2.1122 - acc: 0.96 - ETA: 11:33 - loss: 2.1133 - acc: 0.96 - ETA: 11:33 - loss: 2.1143 - acc: 0.96 - ETA: 11:33 - loss: 2.1123 - acc: 0.96 - ETA: 11:33 - loss: 2.1102 - acc: 0.96 - ETA: 11:33 - loss: 2.1112 - acc: 0.96 - ETA: 11:33 - loss: 2.1154 - acc: 0.96 - ETA: 11:33 - loss: 2.1149 - acc: 0.96 - ETA: 11:33 - loss: 2.1129 - acc: 0.96 - ETA: 11:33 - loss: 2.1124 - acc: 0.96 - ETA: 11:33 - loss: 2.1104 - acc: 0.96 - ETA: 11:33 - loss: 2.1099 - acc: 0.96 - ETA: 11:33 - loss: 2.1082 - acc: 0.96 - ETA: 11:33 - loss: 2.1077 - acc: 0.96 - ETA: 11:33 - loss: 2.1073 - acc: 0.96 - ETA: 11:33 - loss: 2.1068 - acc: 0.96 - ETA: 11:33 - loss: 2.1048 - acc: 0.96 - ETA: 11:33 - loss: 2.1043 - acc: 0.96 - ETA: 11:32 - loss: 2.1038 - acc

 49755/512141 [=>............................] - ETA: 11:25 - loss: 2.0990 - acc: 0.96 - ETA: 11:25 - loss: 2.0973 - acc: 0.96 - ETA: 11:25 - loss: 2.0956 - acc: 0.96 - ETA: 11:24 - loss: 2.0991 - acc: 0.96 - ETA: 11:24 - loss: 2.0986 - acc: 0.96 - ETA: 11:24 - loss: 2.0982 - acc: 0.96 - ETA: 11:24 - loss: 2.0991 - acc: 0.96 - ETA: 11:24 - loss: 2.0987 - acc: 0.96 - ETA: 11:24 - loss: 2.0970 - acc: 0.96 - ETA: 11:24 - loss: 2.0955 - acc: 0.96 - ETA: 11:24 - loss: 2.0951 - acc: 0.96 - ETA: 11:24 - loss: 2.0933 - acc: 0.96 - ETA: 11:24 - loss: 2.0943 - acc: 0.96 - ETA: 11:24 - loss: 2.0952 - acc: 0.96 - ETA: 11:24 - loss: 2.0961 - acc: 0.96 - ETA: 11:24 - loss: 2.0970 - acc: 0.96 - ETA: 11:24 - loss: 2.0979 - acc: 0.96 - ETA: 11:24 - loss: 2.0975 - acc: 0.96 - ETA: 11:24 - loss: 2.0971 - acc: 0.96 - ETA: 11:24 - loss: 2.0955 - acc: 0.96 - ETA: 11:24 - loss: 2.0963 - acc: 0.96 - ETA: 11:23 - loss: 2.0984 - acc: 0.96 - ETA: 11:23 - loss: 2.0992 - acc: 0.96 - ETA: 11:23 - loss: 2.1001 - acc

KeyboardInterrupt: 

In [11]:
# Step 3: Convert the Keras model to TensorFlow Lite model.
sess = tf.keras.backend.get_session()
input_tensor = sess.graph.get_tensor_by_name('input:0')
input_state_tensor = sess.graph.get_tensor_by_name('input_state:0')
output_tensor = sess.graph.get_tensor_by_name('output/Relu:0')
output_state_tensor=sess.graph.get_tensor_by_name('state_out/Reshape:0')

'''converter = tf.lite.TFLiteConverter.from_session(
    sess, [input_tensor,input_state_tensor], [output_tensor,output_state_tensor])'''
converter = tf.lite.TFLiteConverter.from_keras_model_file('tf_lstm_v3.hdf5',
    )
tflite = converter.convert()
open("./TF_LSTM_V3.tflite", "wb").write(tflite)
print('Model converted successfully!')

W0731 13:36:48.833345 13132 deprecation.py:506] From C:\Users\Guo\Anaconda3\envs\tf_1_14\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 13:36:48.837334 13132 rnn_cell.py:246] <tensorflow.lite.experimental.examples.lstm.rnn_cell.TFLiteLSTMCell object at 0x000002BDC381BC18>: Using a concatenated state is slower and will soon be deprecated.  Use state_is_tuple=True.


AttributeError: module 'tensorflow.keras.backend' has no attribute 'track_variable'

In [12]:
open("./TF_LSTM_V3.tflite", "wb").write(tflite)

72164

In [6]:
graph_nodes=[n for n in sess.graph_def.node]

In [5]:
sess = tf.keras.backend.get_session()

In [6]:
graph_nodes

[name: "input"
 op: "Placeholder"
 attr {
   key: "dtype"
   value {
     type: DT_FLOAT
   }
 }
 attr {
   key: "shape"
   value {
     shape {
       dim {
         size: -1
       }
       dim {
         size: 1
       }
       dim {
         size: 3
       }
     }
   }
 }, name: "input_state"
 op: "Placeholder"
 attr {
   key: "dtype"
   value {
     type: DT_FLOAT
   }
 }
 attr {
   key: "shape"
   value {
     shape {
       dim {
         size: -1
       }
       dim {
         size: 128
       }
     }
   }
 }, name: "lambda/transpose/perm"
 op: "Const"
 attr {
   key: "dtype"
   value {
     type: DT_INT32
   }
 }
 attr {
   key: "value"
   value {
     tensor {
       dtype: DT_INT32
       tensor_shape {
         dim {
           size: 3
         }
       }
       tensor_content: "\001\000\000\000\000\000\000\000\002\000\000\000"
     }
   }
 }, name: "lambda/transpose"
 op: "Transpose"
 input: "input"
 input: "lambda/transpose/perm"
 attr {
   key: "T"
   value {
     type

In [7]:
[n.name for n in graph_nodes]

['input',
 'input_state',
 'lambda/transpose/perm',
 'lambda/transpose',
 'lambda/rnn/InputHint-TfLiteDynamicRnn-76e642fab39611e9a06d309c23d28f26-0-None-input',
 'lambda/rnn/Shape',
 'lambda/rnn/strided_slice/stack',
 'lambda/rnn/strided_slice/stack_1',
 'lambda/rnn/strided_slice/stack_2',
 'lambda/rnn/strided_slice',
 'lambda/rnn/Shape_1',
 'lambda/rnn/strided_slice_1/stack',
 'lambda/rnn/strided_slice_1/stack_1',
 'lambda/rnn/strided_slice_1/stack_2',
 'lambda/rnn/strided_slice_1',
 'lambda/rnn/Shape_2',
 'lambda/rnn/strided_slice_2/stack',
 'lambda/rnn/strided_slice_2/stack_1',
 'lambda/rnn/strided_slice_2/stack_2',
 'lambda/rnn/strided_slice_2',
 'lambda/rnn/ExpandDims/dim',
 'lambda/rnn/ExpandDims',
 'lambda/rnn/Const',
 'lambda/rnn/concat/axis',
 'lambda/rnn/concat',
 'lambda/rnn/zeros/Const',
 'lambda/rnn/zeros',
 'lambda/rnn/time',
 'lambda/rnn/TensorArrayV2/element_shape',
 'lambda/rnn/TensorArrayV2',
 'lambda/rnn/TensorArrayV2_1/element_shape',
 'lambda/rnn/TensorArrayV2_1',


In [8]:
model.outputs

[<tf.Tensor 'output/Relu:0' shape=(1, ?, 1) dtype=float32>,
 <tf.Tensor 'state_out/Reshape:0' shape=(?, 128) dtype=float32>]

In [10]:
model.save('tf_lstm_v3.hdf5')